![title.png](https://i.ibb.co/2KmT38V/title.png)

![section-breakpoint.png](https://i.ibb.co/344JqH3/section-breakpoint.png)

![objectives.png](https://i.ibb.co/fxbWnNQ/objectives.png)

![image.png](https://i.ibb.co/rHVSp3w/visual-breakpoint.png)

## Notebook setup and dependency installation

In [96]:
!   pip install -qU \
    openai==1.30 \
    pinecone-client==4.1.0 \
    datasets==2.19 \
    tqdm

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-pinecone 0.1.1 requires pinecone-client<4.0.0,>=3.2.2, but you have pinecone-client 4.1.0 which is incompatible.


In [97]:
from IPython.display import HTML, display, Markdown
from typing import Dict


def chunk_display_html(chunk: Dict[str, str]) -> str:
    html_template = """
<html>
<head>
<style>
    table {{
        font-family: arial, sans-serif;
        border-collapse: collapse;
        width: 100%;
    }}
    td, th {{
        border: 1px solid #dddddd;
        text-align: left;
        padding: 8px;
    }}
</style>
</head>
<body>
    <table>
        <tr>
            <th>Key</th>
            <th>Value</th>
        </tr>
        <tr>
            <td>Title</td>
            <td>{title}</td>
        </tr>
        <tr>
            <td>doi</td>
            <td>{doi}</td>
        </tr>
        <tr>
            <td>Chunk ID</td>
            <td>{chunk_id}</td>
        </tr>
        <tr>
            <td>chunk</td>
            <td>{chunk}</td>
        </tr>
        <tr>
            <td>id</td>
            <td>{id}</td>
        </tr>
        <tr>
            <td>Summary</td>
            <td>{summary}</td>
        </tr>
        <tr>
            <td>Source</td>
            <td>{source}</td>
        </tr>
        <tr>
            <td>Authors</td>
            <td>{authors}</td>
        </tr>
        <tr>
            <td>Categories</td>
            <td>{categories}</td>
        </tr>
        <tr>
            <td>Comment</td>
            <td>{comment}</td>
        </tr>
        <tr>
            <td>Journal Reference</td>
            <td>{journal_ref}</td>
        </tr>
        <tr>
            <td>Primary Category</td>
            <td>{primary_category}</td>
        </tr>
        <tr>
            <td>Published</td>
            <td>{published}</td>
        </tr>
        <tr>
            <td>Updated</td>
            <td>{updated}</td>
        </tr>
        <tr>
            <td>References</td>
            <td>{references}</td>
        </tr>
    </table>
</body>
</html>
"""

    # Format the HTML with the generated rows
    html_output = html_template.format(
        doi=chunk.get("doi", "N/A"),
        chunk_id=chunk.get("chunk-id", "N/A"),
        chunk=chunk.get("chunk", "N/A"),
        id=chunk.get("id", "N/A"),
        title=chunk.get("title", "N/A"),
        summary=chunk.get("summary", "N/A"),
        source=chunk.get("source", "N/A"),
        authors=chunk.get("authors", "N/A"),
        categories=chunk.get("categories", "N/A"),
        comment=chunk.get("comment", "N/A"),
        journal_ref=chunk.get("journal_ref", "N/A"),
        primary_category=chunk.get("primary_category", "N/A"),
        published=chunk.get("published", "N/A"),
        updated=chunk.get("updated", "N/A"),
        references=chunk.get("references", "N/A"),
    )

    # Display the HTML in an IPython notebook
    display(HTML(html_output))


def display_retrieved_context(context_response):
    # HTML template for the main container and individual tables
    html_template = """
    <html>
    <head>
    <style>
        .container {{
            display: flex;
            flex-wrap: wrap;
        }}
        .table-container {{
            margin: 10px;
            padding: 10px;
            border: 1px solid #dddddd;
        }}
        table {{
            font-family: arial, sans-serif;
            border-collapse: collapse;
            width: 100%;
        }}
        td, th {{
            border: 1px solid #dddddd;
            text-align: left;
            padding: 8px;
        }}
    </style>
    </head>
    <body>
        <div class="container">
            {tables}
        </div>
    </body>
    </html>
    """

    # Function to generate HTML table for a single dictionary
    def generate_table_for_dict(data):
        rows = "\n".join(
            "<tr><td>{key}</td><td>{value}</td></tr>".format(
                key=key, value=value if value is not None else "N/A"
            )
            for key, value in data.items()
        )
        table_html = """
        <div class="table-container">
            <table>
                <tr>
                    <th>Key</th>
                    <th>Value</th>
                </tr>
                {rows}
            </table>
        </div>
        """.format(
            rows=rows
        )
        return table_html

    # Generate HTML tables for all dictionaries in the list
    tables = "\n".join(
        generate_table_for_dict(data["metadata"]) for data in context_response
    )

    # Format the main HTML with the generated tables
    html_output = html_template.format(tables=tables)

    # Display the HTML in an IPython notebook
    display(HTML(html_output))


def display_markdown(content: str) -> None:
    display(Markdown(content))

![image.png](https://i.ibb.co/rHVSp3w/visual-breakpoint.png)

![step-away-rag.png](https://i.ibb.co/Y288TWR/step-away-rag.png)

## Setup OpenAI

Enter the OpenAI API key and instantiate the OpenAI clinet.

Copy this openai API key `sk-XXXXXXXXXX` into the prompt.

In [90]:
import getpass
from openai import OpenAI

OPENAI_API_KEY = getpass.getpass("Enter your OpenAI API key: ")
openai = OpenAI(api_key=OPENAI_API_KEY)

![section-breakpoint.png](https://i.ibb.co/344JqH3/section-breakpoint.png)

## Implement request to OpenAI GPT Models


Implement a function that will send a prompt to an LLM and return an answer.  
The OpenAI client has the following signature:  
```python 
openai_client.chat.completions.create(model: str, messages=List[Dict[str, str]])
```

API reference: https://platform.openai.com/docs/api-reference/chat/create 

In [91]:
def llm_completion(prompt: str, openai_client: OpenAI, model: str = "gpt-3.5-turbo") -> str:

    response = openai_client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a helpful assistant. Output is markdown"},
            {"role": "user", "content": prompt},
        ],
        temperature=0.0,
    )

    return response.choices[0].message.content


![section-breakpoint.png](https://i.ibb.co/344JqH3/section-breakpoint.png)

In [92]:
display_markdown(llm_completion("What is the capital of Germany?", openai))

The capital of Germany is Berlin.

In [93]:
display_markdown(llm_completion("Who is 25th person that landed on the moon?", openai, model="gpt-3.5-turbo"))

The 25th person to land on the moon was astronaut Harrison Schmitt. He was a member of the Apollo 17 mission in December 1972 and was the only geologist to walk on the moon.

![hallucinations-problem.png](https://i.ibb.co/gMvNZC6/hallucinations-problem.png)

![section-breakpoint.png](https://i.ibb.co/344JqH3/section-breakpoint.png)

In [94]:
display_markdown(llm_completion("What are key benefits of mistral 7B?", openai, model="gpt-3.5-turbo"))

The Mistral 7B is a powerful and versatile wind turbine that offers several key benefits:

1. **High Efficiency**: The Mistral 7B is designed to maximize energy production with its high-efficiency blades and generator, ensuring that it can generate a significant amount of electricity from the wind.

2. **Robust Construction**: This wind turbine is built to withstand harsh weather conditions and has a durable construction that ensures long-term reliability and performance.

3. **Quiet Operation**: The Mistral 7B is designed to operate quietly, making it suitable for residential areas or locations where noise levels need to be minimized.

4. **Easy Installation**: With its user-friendly design and easy installation process, the Mistral 7B can be set up quickly and efficiently, saving time and effort.

5. **Remote Monitoring**: The Mistral 7B can be equipped with remote monitoring capabilities, allowing users to track its performance and troubleshoot any issues from a distance.

Overall, the Mistral 7B offers a combination of efficiency, durability, and ease of use, making it a reliable choice for renewable energy generation.

![knowledge-cutoff.png](https://i.ibb.co/ccccpxZ/knowledge-cutoff.png)

![section-breakpoint.png](https://i.ibb.co/344JqH3/section-breakpoint.png)

In [95]:
context_example = """
Answer the question based on the following context. If you don't can't find the answer, tell I don't know.

Context:
We introduce Mistral 7B, a 7–billion-parameter language model engineered for
superior performance and efficiency. Mistral 7B outperforms the best open 13B
model (Llama 2) across all evaluated benchmarks, and the best released 34B
model (Llama 1) in reasoning, mathematics, and code generation. Our model
leverages grouped-query attention (GQA) for faster inference, coupled with sliding
window attention (SWA) to effectively handle sequences of arbitrary length with a
reduced inference cost. We also provide a model fine-tuned to follow instructions,
Mistral 7B – Instruct, that surpasses Llama 2 13B – chat model both on human and
automated benchmarks. Our models are released under the Apache 2.0 license.

Question: What are key benefits of Mistral 7B?
"""
display_markdown(llm_completion(context_example, openai))

The key benefits of Mistral 7B include:

1. Superior performance and efficiency compared to other models.
2. Outperforming the best open 13B model (Llama 2) across all evaluated benchmarks.
3. Outperforming the best released 34B model (Llama 1) in reasoning, mathematics, and code generation.
4. Leveraging grouped-query attention (GQA) for faster inference.
5. Using sliding window attention (SWA) to effectively handle sequences of arbitrary length with reduced inference cost.
6. Providing a fine-tuned model, Mistral 7B – Instruct, that surpasses Llama 2 13B – chat model on both human and automated benchmarks.

![visual-breakpoint.png](https://i.ibb.co/rHVSp3w/visual-breakpoint.png)

![why-rag.png](https://i.ibb.co/KF3xr64/why-rag.png)

![section-breakpoint.png](https://i.ibb.co/344JqH3/section-breakpoint.png)

![how-to-build-rag.png](https://i.ibb.co/wgXqfFv/how-to-build-rag.png)

![visual-breakpoint.png](https://i.ibb.co/rHVSp3w/visual-breakpoint.png)

## 1. Build a knowledge base

![section-breakpoint.png](https://i.ibb.co/344JqH3/section-breakpoint.png)

![build-knowledge-base.png](https://i.ibb.co/dGnjrCk/build-knowledge-base.png)


![section-breakpoint.png](https://i.ibb.co/344JqH3/section-breakpoint.png)

### Setup Pinecone 

Enter the Pinecone API key inside the prompt and create a Pinecone client.

In [55]:
PINECONE_REGION = "eu-west-1"
PINECONE_CLOUD = "aws"
INDEX_NAME = "pinecone-workshop-1"
VECTOR_DIMENSIONS = 1536
PINECONE_API_KEY = getpass.getpass("Enter your Pinecone API key: ")

In [56]:
from pinecone import Pinecone

pinecone = Pinecone(api_key=PINECONE_API_KEY)

pinecone.list_indexes()

{'indexes': [{'dimension': 1536,
              'host': 'pinecone-workshop-1-2kw20wn.svc.apu-57e2-42f6.pinecone.io',
              'metric': 'cosine',
              'name': 'pinecone-workshop-1',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'eu-west-1'}},
              'status': {'ready': True, 'state': 'Ready'}},
             {'dimension': 1536,
              'host': 'pinecone-worshop-1-2kw20wn.svc.apw5-4e34-81fa.pinecone.io',
              'metric': 'cosine',
              'name': 'pinecone-worshop-1',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-west-2'}},
              'status': {'ready': True, 'state': 'Ready'}}]}

![section-breakpoint.png](https://i.ibb.co/344JqH3/section-breakpoint.png)

### Create a Pinecone Index

Create a Serverless Index with OpenAI embeddings size with cosine similarity metrics. 
The index creation requires index name, dimension of embeddings, similarity metric and serverless spec for a serverless setup.  

Pseudo code:  
```python
pinecone.create_index(
    name=<add-index-name>,
    dimension=<add-vector-dimension>,
    metric=<add-similarity-metric>,
    spec=ServerlessSpec(cloud=<add-cloud-name>, region=<add-region-name>)
)
```

More info o serverless: https://docs.pinecone.io/reference/architecture/serverless-architecture#overview  
Ref API: https://docs.pinecone.io/reference/api/control-plane/create_index 

In [57]:
from pinecone import ServerlessSpec

# Check if the index already exists and delete it
if INDEX_NAME in [index.name for index in pinecone.list_indexes()]:
    pinecone.delete_index(INDEX_NAME)

# TODO: Create a new index with the specified name, dimension, metric, and spec
# Use `pinecone.create_index(name=, dimension=, metric=, spec=ServerlessSpec(cloud=, region=)`
pinecone.create_index(
    name=INDEX_NAME,
    dimension=VECTOR_DIMENSIONS,
    metric="cosine",
    spec=ServerlessSpec(cloud=PINECONE_CLOUD, region=PINECONE_REGION),
)

# Create a new Index reference object with the specified name and pool_threads
index = pinecone.Index(INDEX_NAME, pool_threads=20)
print(index.describe_index_stats())

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}


![section-breakpoint.png](https://i.ibb.co/344JqH3/section-breakpoint.png)

### Dataset

We are going to use a sample of 1000 AI papers that can be found here: https://huggingface.co/datasets/smartcat/ai-arxiv2-chunks-embedded 

The data set is already chunked and encoded using `text-embeddings-3-small` so we can just load and upsert it to the Pinecone.

If you want to play with chunking strategies and embeddings, you can find the full data set here: https://huggingface.co/datasets/jamescalam/ai-arxiv2

Dataset API reference: https://huggingface.co/docs/datasets/en/index  
Slicing and indexing: https://huggingface.co/docs/datasets/en/access 

In [58]:
import datasets

dataset = datasets.load_dataset("smartcat/ai-arxiv2-chunks-embedded", split="train")
dataset

Dataset({
    features: ['doi', 'chunk-id', 'chunk', 'id', 'title', 'summary', 'source', 'authors', 'categories', 'comment', 'journal_ref', 'primary_category', 'published', 'updated', 'references', 'embeddings', 'metadata'],
    num_rows: 79782
})

In [59]:
chunk_display_html(dataset[0])

Key,Value
Title,Foundations of Vector Retrieval
DOI,2401.09350
Chunk ID,0
Chunk,"4 2 0 2 n a J 7 1 ] S D . s c [ 1 v 0 5 3 9 0 . 1 0 4 2 : v i X r a Sebastian Bruch # Foundations of Vector Retrieval # Preface We are witness to a few years of remarkable developments in Artificial Intelligence with the use of advanced machine learning algorithms, and in particular, deep learning. Gargantuan, complex neural networks that can learn through self-supervisionâand quickly so with the aid of special- ized hardwareâtransformed the research landscape so dramatically that, incremental overnight it seems, many fields experienced not the usual, progress, but rather a leap forward. Machine translation, natural language understanding, information retrieval, recommender systems, and computer vision are but a few examples of research areas that have had to grapple with the shock. Countless other disciplines beyond computer science such as robotics, biology, and chemistry too have benefited from deep learning."
ID,2401.09350#0
Summary,"Vectors are universal mathematical objects that can represent text, images, speech, or a mix of these data modalities. That happens regardless of whether data is represented by hand-crafted features or learnt embeddings. Collect a large enough quantity of such vectors and the question of retrieval becomes urgently relevant: Finding vectors that are more similar to a query vector. This monograph is concerned with the question above and covers fundamental concepts along with advanced data structures and algorithms for vector retrieval. In doing so, it recaps this fascinating topic and lowers barriers of entry into this rich area of research."
Source,http://arxiv.org/pdf/2401.09350
Authors,Sebastian Bruch
Categories,"cs.DS, cs.IR"
Comment,None


In [60]:
print(len(dataset[0]["embeddings"]))
print(dataset[0]["embeddings"][:10])

1536
[-0.020014504, -0.013545036, 0.04353385, -0.0029185074, 0.03552278, -0.034943033, 0.013927143, 0.06566971, -0.06888468, 0.026971487]


In [61]:
list(dataset[0]["metadata"].keys())

['authors',
 'chunk_id',
 'doc_id',
 'primary_category',
 'published',
 'source',
 'summary',
 'text',
 'title']

![section-breakpoint.png](https://i.ibb.co/344JqH3/section-breakpoint.png)

### Data upsert to Pinecone

Let insert data to the Pinecone in batches. 

From our data set we need 3 columns:
1. `id` - the ID of the chunk we want to insert
2. `embeddings` - contains a vector embedding of the chunk. It uses `text-embeddings-3-small`
3. `metadata` - a dictinary with additional data about the chunk. 

The code for upserting:
```python
index.upsert(vectors=[ 
    (id1, vector1, metadata1),
    (id2, vector2, metadata2),
    ....
 ])
```

### Note on optimization

To improve througput, we are going to add `async_req=True` parameter. 
Upsert will return futures that we need to wait.

Optimization tips:
1. Deploy application at the same region
2. Use batching upsert
3. Use parallelized upsert
4. Use GRPCIndex, but make sure to add backoff for throttling
5. Use namespaces and metadata filtering
6. Avoid quotas and limits: https://docs.pinecone.io/reference/quotas-and-limits

Optimized upsert code:
```python
future = index.upsert(vectors=[ 
    (id1, vector1, metadata1),
    (id2, vector2, metadata2),
    ....
    ],
    async_req=True
)
....
future.get() # wait for upsert to complete
```

For scale-up and optimizations make sure to read: : https://docs.pinecone.io/guides/operations/performance-tuning#increasing-throughput  
Metadata filtering: Metadata filtering: https://docs.pinecone.io/guides/data/filter-with-metadata 

In [66]:
from tqdm import tqdm
from pinecone import Index

def upsert_batch(ds: datasets.Dataset, index: Index, batch_size: int = 100) -> None:
    futures = []
    for i in range(0, len(ds), batch_size):
        i_end = min(i + batch_size, len(ds))
        batch = ds.select(range(i, i_end))

        # The upsert requires the vectors to be a list of tuples (id, vector, metadata)
        # Example: [(id1, vector1, metadata1), (id2, vector2, metadata2), ...]
        # TODO: Select the id, embeddings, and metadata from the batch
        ids = batch["id"]
        embeddings = batch["embeddings"]
        metadata = batch["metadata"]

        # TODO: Upsert the vectors to the Pinecone index asynchronously
        # Use `index.upsert(vectors=[(id1, vector1, metadata1), (id2, vector2, metadata2)], async_req=True)`
        futures.append(
            index.upsert(vectors=list(zip(ids, embeddings, metadata)), async_req=True)
        )

    # Wait for all the upserts to complete
    for future in tqdm(futures, total=len(futures), desc="Upsert to Pinecone"):
        future.get()

In [67]:
upsert_batch(dataset.select(range(500)), index, batch_size=20)

Upsert to Pinecone: 100%|██████████| 25/25 [00:02<00:00,  9.95it/s]


In [69]:
print(index.describe_index_stats())

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 500}},
 'total_vector_count': 500}


In [88]:
upsert_batch(dataset.select(range(40_000)), index)

Upsert to Pinecone: 100%|██████████| 400/400 [02:44<00:00,  2.43it/s]


In [89]:
print(index.describe_index_stats())

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 42500}},
 'total_vector_count': 42500}


![section-breakpoint.png](https://i.ibb.co/344JqH3/section-breakpoint.png)

### What has been done with data set?

![chunking-dataset.png](https://i.ibb.co/9wV70Q7/chunking-dataset.png)

![section-breakpoint.png](https://i.ibb.co/344JqH3/section-breakpoint.png)

#### Chunking Strategies
1. Character split (with overlapping)
2. Recursive character split
3. Document specific splitting
4. Semantic Chunking
5. Agentic?
6. More?

Introduction to chunking: https://github.com/FullStackRetrieval-com/RetrievalTutorials/blob/main/tutorials/LevelsOfTextSplitting/5_Levels_Of_Text_Splitting.ipynb


![visual-breakpoint.png](https://i.ibb.co/rHVSp3w/visual-breakpoint.png)

## Revisit Agenda


![done-next.png](https://i.ibb.co/QYC9nrb/done-next.png)


![visual-breakpoint.png](https://i.ibb.co/rHVSp3w/visual-breakpoint.png)

## 2. Retrieve against the query



![section-breakpoint.png](https://i.ibb.co/344JqH3/section-breakpoint.png)

![semantic-search.png](https://i.ibb.co/2dWGRHn/semantic-search.png)

![section-breakpoint.png](https://i.ibb.co/344JqH3/section-breakpoint.png)

Once we inserted everythong to Pinecone, let's query it. The input is query text and the output is the top similar chunks.
Steps: 
1. Encode the input text to generate embeddings 
2. Call Pinecone's query function to retrieve top K similar results

### Encode (create embeddings)

Create embeddings:
```python
openai_client.embeddings.create(model="embedding-model-name", input="Text to create embeddings")
```

Embedding API Ref: https://platform.openai.com/docs/api-reference/embeddings  
Query API Ref: https://docs.pinecone.io/reference/api/data-plane/query 

In [71]:
from typing import List


def encode(
    text: str, openai_client: OpenAI, model: str = "text-embedding-3-small"
) -> List[float]:
    # TODO: create embeddings using OpenAI API
    # 1. Call openai_client.embeddings.create with the model and text
    response = openai_client.embeddings.create(
        model=model, input=text
    )

    return response.data[0].embedding

In [72]:
res = encode("What are key benefits of mistral 7B?", openai)
print(res[:10])
print(len(res))

[-0.02581052854657173, 0.03242715075612068, 0.019921164959669113, -0.021119002252817154, -0.036020658910274506, 0.023999514058232307, -0.05059434100985527, 0.05886511504650116, 0.024940671399235725, -0.006969555746763945]
1536


![section-breakpoint.png](https://i.ibb.co/344JqH3/section-breakpoint.png)

### Query Pinecone

Query Pinecone:
We are going to retrieve metadata, but not vectors itself
```python
index.query(
        vector=QUERY_EMBEDDINGS, top_k=TOP_K, include_metadata=True, include_values=False
    )
```

In [73]:
from pinecone import QueryResponse


def semantic_search(
    query: str, index: Index, openai_client: OpenAI, top_k: int = 10
) -> QueryResponse:
    # TODO: Implement semantic search using the OpenAI API and Pinecone index
    # 1. Reuse `encode` function to create embeddings for the query
    # 2. Use `index.query` to query the index with the vector and top_k and set include_metadata=True
    query_embedding = encode(query, openai_client)
    ret = index.query(
        vector=query_embedding, top_k=top_k, include_metadata=True, include_values=False
    )
    return ret

In [74]:
res = semantic_search("What is Mistral 7B?", index, openai)
print(res.matches[0].metadata['text'])

Abstract
We introduce Mistral 7B, a 7âbillion-parameter language model engineered for superior performance and efficiency. Mistral 7B outperforms the best open 13B model (Llama 2) across all evaluated benchmarks, and the best released 34B model (Llama 1) in reasoning, mathematics, and code generation. Our model leverages grouped-query attention (GQA) for faster inference, coupled with sliding window attention (SWA) to effectively handle sequences of arbitrary length with a reduced inference cost. We also provide a model fine-tuned to follow instructions, Mistral 7B â Instruct, that surpasses Llama 2 13B â chat model both on human and automated benchmarks. Our models are released under the Apache 2.0 license. Code: https://github.com/mistralai/mistral-src Webpage: https://mistral.ai/news/announcing-mistral-7b/
# Introduction


![visual-breakpoint.png](https://i.ibb.co/rHVSp3w/visual-breakpoint.png)

## 3. Augment and generate

![section-breakpoint.png](https://i.ibb.co/344JqH3/section-breakpoint.png)

![workflow-rag-simple.png](https://i.ibb.co/p0gwY23/workflow-rag-simple.png)

![section-breakpoint.png](https://i.ibb.co/344JqH3/section-breakpoint.png)

### Generate a final response

Combine all pieces together:
1. Perform a semantic search on input query
2. Build a context (prompt) for a LLM
3. Call LLM to generate a final response
4. Return a final response and retrieved context

The relevant context can be found in metadata, you can use:
1. `title` - a Paper title
2. `published` - a publish date
3. `primart_category` 
4. `summary` - a paper summary
5. `text` - a chunk text - this is the most useful info to build a context for LLM

In [98]:
from typing import Tuple

def from_metadata(metadata: Dict) -> str:
    return f"""
***
    Title: {metadata['title']}
    Authors: {metadata['authors'][:5]}
    Published: {metadata['published']}
    Primary category: {metadata['primary_category']}
    Paper summary: {metadata['summary']}
    Text: {metadata['text']}
"""


def get_prompt(query: str, query_results: QueryResponse) -> str:
    context = "\n".join(
        [from_metadata(result.metadata) for result in query_results.matches]
    )
    return f"""
Answer the question based on the following context. If you don't can't find the answer, tell I don't know.
The answers should be clear, easy to understand, complete and comprehensive.

Context:
{context}

Question: {query}
"""


def rag(query: str, index: Index, openai_client: OpenAI, top_k: int = 5) -> Tuple[str, QueryResponse]:
    # TODO: Wire all RAG pieces together to generate a response (retrieve, augment, generate)
    # 1. [RETRIEVE]: Reuse `semantic_search` function to get the top_k results
    # 2. [AUGMENT]: Use `get_prompt` function to generate the prompt (context + question)
    # 3. [GENERATE]: Use `llm_completion` function to generate the response
    # 4. Return the response and query_results
    query_results = semantic_search(query, index, openai_client, top_k=top_k)
    prompt = get_prompt(query, query_results)
    response = llm_completion(prompt, openai_client)
    return response, query_results

In [99]:
answer, context = rag("What are key benefits of Mistral 7B?", index, openai)
display_markdown(answer)

The key benefits of Mistral 7B are:

1. **Superior Performance**: Mistral 7B is engineered for superior performance, outperforming the Llama 2 13B model across all evaluated benchmarks and surpassing the Llama 1 34B model in reasoning, mathematics, and code generation.

2. **Efficiency**: Mistral 7B is designed to be efficient, balancing high performance with reduced computational costs and inference latency. This efficiency is crucial for deployment in practical, real-world scenarios.

3. **Innovative Attention Mechanisms**: Mistral 7B leverages grouped-query attention (GQA) for faster inference and sliding window attention (SWA) to handle sequences of arbitrary length effectively with reduced inference costs. These attention mechanisms contribute to the model's enhanced performance and efficiency.

4. **Ease of Deployment**: Mistral 7B is released under the Apache 2.0 license, accompanied by a reference implementation that facilitates easy deployment either locally or on cloud platforms such as AWS, GCP, or Azure. Integration with tools like vLLM inference server and Hugging Face is streamlined for easier deployment.

5. **Fine-Tuning Capabilities**: Mistral 7B can be fine-tuned across a myriad of tasks, showcasing adaptability and superior performance. Models fine-tuned from Mistral 7B, such as Mistral 7B -- Instruct, have demonstrated significant improvements over existing models in both human and automated benchmarks.

Overall, Mistral 7B aims to provide a high-performing, efficient language model that can be used in a wide range of real-world applications, offering advancements in performance, efficiency, and ease of deployment.

In [100]:
display_retrieved_context(context.matches[:2])

Key,Value
authors,"Albert Q. Jiang, Alexandre Sablayrolles, Arthur Mensch, Chris Bamford, Devendra Singh Chaplot, Diego de las Casas, Florian Bressand, Gianna Lengyel, Guillaume Lample, Lucile Saulnier, Lélio Renard Lavaud, Marie-Anne Lachaux, Pierre Stock, Teven Le Scao, Thibaut Lavril, Thomas Wang, Timothée Lacroix, William El Sayed"
chunk_id,3.0
doc_id,2310.06825
primary_category,cs.CL
published,20231010
source,http://arxiv.org/pdf/2310.06825
summary,"We introduce Mistral 7B v0.1, a 7-billion-parameter language model engineered for superior performance and efficiency. Mistral 7B outperforms Llama 2 13B across all evaluated benchmarks, and Llama 1 34B in reasoning, mathematics, and code generation. Our model leverages grouped-query attention (GQA) for faster inference, coupled with sliding window attention (SWA) to effectively handle sequences of arbitrary length with a reduced inference cost. We also provide a model fine-tuned to follow instructions, Mistral 7B -- Instruct, that surpasses the Llama 2 13B -- Chat model both on human and automated benchmarks. Our models are released under the Apache 2.0 license."
text,"Mistral 7B is released under the Apache 2.0 license. This release is accompanied by a reference implementation1 facilitating easy deployment either locally or on cloud platforms such as AWS, GCP, or Azure using the vLLM [17] inference server and SkyPilot 2. Integration with Hugging Face 3 is also streamlined for easier integration. Moreover, Mistral 7B is crafted for ease of fine-tuning across a myriad of tasks. As a demonstration of its adaptability and superior performance, we present a chat model fine-tuned from Mistral 7B that significantly outperforms the Llama 2 13B â Chat model. Mistral 7B takes a significant step in balancing the goals of getting high performance while keeping large language models efficient. Through our work, our aim is to help the community create more affordable, efficient, and high-performing language models that can be used in a wide range of real-world applications. # 2 Architectural details The cat sat on the The cat sat on the window size â_ââ> The cat sat on the Vanilla Attention Sliding Window Attention Effective Context Length"
title,Mistral 7B
Key,Value


![section-breakpoint.png](https://i.ibb.co/344JqH3/section-breakpoint.png)

In [101]:
answer, _ = rag("The difference between Mistral and Kosmos?", index, openai, top_k=20)
display_markdown(answer)

- **Mistral 7B** is a 7-billion-parameter language model engineered for superior performance and efficiency. It outperforms Llama 2 13B across all evaluated benchmarks and Llama 1 34B in reasoning, mathematics, and code generation. Mistral 7B leverages grouped-query attention (GQA) for faster inference and sliding window attention (SWA) to handle sequences of arbitrary length efficiently. Additionally, Mistral 7B provides a model fine-tuned to follow instructions, named Mistral 7B -- Instruct, which surpasses the Llama 2 13B -- Chat model on both human and automated benchmarks. The models are released under the Apache 2.0 license.

- **Kosmos-1** is a Multimodal Large Language Model (MLLM) designed to perceive general modalities, learn in context (few-shot learning), and follow instructions (zero-shot learning). It is trained from scratch on web-scale multimodal corpora, including text and images, image-caption pairs, and text data. Kosmos-1 excels in language understanding, generation, OCR-free NLP, perception-language tasks, and vision tasks. It can benefit from cross-modal transfer and has a dataset for diagnosing nonverbal reasoning capability.

- **Difference between Mistral and Kosmos**: Mistral 7B is a language model focused on performance and efficiency, leveraging specific attention mechanisms for inference. On the other hand, Kosmos-1 is a Multimodal Large Language Model that integrates perception with language models, excelling in various tasks across different modalities and learning contexts.

![visual-breakpoint.png](https://i.ibb.co/rHVSp3w/visual-breakpoint.png)

## Bonus Topics

![section-breakpoint.png](https://i.ibb.co/344JqH3/section-breakpoint.png)

TBA - Add mini-agenda

![section-breakpoint.png](https://i.ibb.co/344JqH3/section-breakpoint.png)

### LangChain

Once we explored the simple components of RAG, we can use existing frameworks to build them.  

Langchain: https://www.langchain.com/langchain 

In [80]:
!pip install -qU \
    langchain-pinecone \
    langchain-openai \
    langchain

In [81]:
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQAWithSourcesChain

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key=OPENAI_API_KEY,
)

vectorstore = PineconeVectorStore(
    index=index, embedding=embeddings, text_key="text", pinecone_api_key=PINECONE_API_KEY, index_name=INDEX_NAME,)

llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo", temperature=0.0)
langchain_rag = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever()
)

In [82]:
langchain_rag.invoke("What are all benefits of using Mistral 7B?")

{'question': 'What are all benefits of using Mistral 7B?',
 'answer': 'The benefits of using Mistral 7B include being released under the Apache 2.0 license, easy deployment on cloud platforms, integration with Hugging Face, ease of fine-tuning across tasks, and superior performance compared to other models.\n',
 'sources': 'http://arxiv.org/pdf/2310.06825'}

![visual-breakpoint.png](https://i.ibb.co/rHVSp3w/visual-breakpoint.png)

### Fine-Tune vs RAG vs Prompt Engineering

![rag-vs-prompt-vs-finetune.png](https://i.ibb.co/C8rvnTY/Screenshot-2024-05-30-at-21-17-45.png)

![visual-breakpoint.png](https://i.ibb.co/rHVSp3w/visual-breakpoint.png)

### Advanced RAG

In [83]:
answer, context = rag(
    "The key difference between Mistral 7B and Palm?", index, openai, top_k=20
)
display_markdown(answer)

The key difference between Mistral 7B and Palm lies in their respective focuses and capabilities:

- **Mistral 7B**:
  - **Focus**: Mistral 7B is engineered for superior performance and efficiency in natural language processing tasks, including reasoning, mathematics, and code generation.
  - **Model Features**: It leverages grouped-query attention (GQA) for faster inference and sliding window attention (SWA) to handle sequences of arbitrary length efficiently.
  - **Fine-Tuning**: Mistral 7B offers a model fine-tuned to follow instructions, Mistral 7B -- Instruct, which surpasses the Llama 2 13B -- Chat model on both human and automated benchmarks.
  - **License**: Mistral 7B models are released under the Apache 2.0 license.

- **Palm**:
  - **Focus**: Palm models aim to be state-of-the-art language models with better multilingual and reasoning capabilities, as well as improved compute efficiency.
  - **Training**: Palm 2 is a Transformer-based model trained using a mixture of objectives to achieve high-quality performance on downstream tasks.
  - **Inference**: Palm models exhibit faster and more efficient inference compared to their predecessor, Palm.
  - **Responsibility**: Palm models demonstrate stable performance on responsible AI evaluations and enable inference-time control over toxicity without additional overhead.

In summary, Mistral 7B is focused on performance and efficiency in NLP tasks, while Palm models emphasize multilingual capabilities, reasoning, and responsible AI practices.

#### Indexing Time

#### Query Time

#### Evaluations

#### Agentic Approach

![visual-breakpoint.png](https://i.ibb.co/rHVSp3w/visual-breakpoint.png)

## Resources

In [ ]:
# Notes
# Better embeddings
# Better chunking
# Better indexing
# Better query understanding
# History management
# Evaluation
# Ranking (Context Limitation)
# Guardrails
# Frameworks (langchain / canopy)

![visual-breakpoint.png](https://i.ibb.co/rHVSp3w/visual-breakpoint.png)